In [9]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
from dotenv import load_dotenv

load_dotenv()

True

## Set up models

In [7]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct')
colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(rm=colbertv2, lm=turbo)

# Data

In [3]:
train = [('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', 'Kevin Greutert'),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', 'Foxcatcher'),
         ('In what year was the star of To Hell and Back born?', '1925'),
         ('Which award did the first book of Gary Zukav receive?', 'U.S. National Book Award'),
         ('What documentary about the Gilgo Beach Killer debuted on A&E?', 'The Killing Season'),
         ('Which author is English: John Braine or Studs Terkel?', 'John Braine'),
         ('Who produced the album that included a re-recording of "Lithium"?', 'Butch Vig')]

train = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in train]

In [4]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', 'E. L. Doctorow'),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', 'Gainesville, Florida'),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', '1828'),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', 'Andre Dirrell'),
       ('The sports nutrition business established by Oliver Cookson is based in which county in the UK?', 'Cheshire'),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', 'February 13, 1980'),
       ('Kyle Moran was born in the town on what river?', 'Castletown River'),
       ("The actress who played the niece in the Priest film was born in what city, country?", 'Surrey, England'),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', 'Portrait of a Marriage'),
       ('What year was the father of the Princes in the Tower born?', '1442'),
       ('What river is near the Crichton Collegiate Church?', 'the River Tyne'),
       ('Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?', 'Renault'),
       ('André Zucca was a French photographer who worked with a German propaganda magazine published by what Nazi organization?', 'the Wehrmacht')]

dev = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in dev]

# Evaluation

In [14]:
NUM_THREADS = 32
evaluate_hotpot = Evaluate(devset=dev, metric=metric_EM, num_threads=NUM_THREADS, display_progress=True, display_table=15)

# Chain of Thought

In [5]:
class CoT(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()

        # here we declare the chain of thought sub-module, so we can later compile it (e.g., teach it a prompt)
        self.generate_answer = dspy.ChainOfThought('question -> answer')
    
    def forward(self, question):
        return self.generate_answer(question=question)  # here we use the module

In [10]:
metric_EM = dspy.evaluate.answer_exact_match

optimizer = BootstrapFewShot(metric=metric_EM, max_bootstrapped_demos=2)
cot_compiled = optimizer.compile(CoT(), trainset=train)

 29%|█████████████████████████████████████████▏                                                                                                      | 2/7 [00:02<00:05,  1.12s/it]

Bootstrapped 2 full traces after 3 examples in round 0.


In [11]:
turbo.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Who produced the album that included a re-recording of "Lithium"?
Answer: Butch Vig

---

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

---

Question: Which author is English: John Braine or Studs Terkel?
Answer: John Braine

---

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Answer: Kevin Greutert

---

Question: In what year was the star of To Hell and Back born?
Answer: 1925

---

Question: What documentary about the Gilgo Beach Killer debuted on A&E?
Answer: The Killing Season

---

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Reasoning: Let's think step by step in order to find the answer. We know that the Du Pont fami

In [13]:
evaluate_hotpot(cot_compiled)

Average Metric: 3 / 13  (23.1): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  8.45it/s]
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' 'False' 'False' '✔️ [True]' '✔️ [True]' 'False' 'False'
 'False' 'False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


Average Metric: 3 / 13  (23.1%)


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,"find the answer. We know that E. L. Doctorow is a novelist, editor, and professor, while Julia Peterkin is a novelist and short story writer....",E. L. Doctorow,✔️ [True]
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida","find the answer. We know that the song ""Right Back At It Again"" was released in 2013, so we can look up the songs that...","Ocala, Florida",False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,find the answer. We know that the 1971 San Francisco mayoral election was held in 1971. We can then look up the winner of that...,1969,False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,"find the answer. We know that Anthony Dirrell is a professional boxer, and specifically a super middleweight. We can then look up the current super...",Andre Dirrell,✔️ [True]
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,find the answer. We know that Oliver Cookson is the founder of a sports nutrition business. We can then look up the location of the...,Cheshire,✔️ [True]
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980",find the answer. We know that the actor played roles in both First Wives Club and Searching for the Elephant. We can then look up...,"September 18, 1961",False
6,Kyle Moran was born in the town on what river?,Castletown River,"find the answer. We know that Kyle Moran was born in a town, so we can look up the towns that he could have been...",Hudson River,False
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England","find the answer. We know that the actress who played the niece in the Priest film is a well-known actress, so we can look up...","London, England",False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,"find the answer. We know that Noel Harrison is an actor, and that he has a daughter. We can then look up the movies that...",The Trials of Oscar Wilde,False
9,What year was the father of the Princes in the Tower born?,1442,"find the answer. We know that the Princes in the Tower were born in 1470 and 1473, and their father was born when he was...",1457,False


23.08

# RAG

In [17]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        # declare three modules: the retriever, a query generator, and an answer generator
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought("question -> search_query")
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        # generate a search query from the question, and use it to retrieve passages
        search_query = self.generate_query(question=question).search_query
        passages = self.retrieve(search_query).passages

        # generate an answer from the passages and the question
        return self.generate_answer(context=passages, question=question)

Out of curiosity, evluate the uncompiled or zero-shot version of this

In [18]:
evaluate_hotpot(RAG(), display_table=0)

Average Metric: 2 / 13  (15.4): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  3.45it/s]


Average Metric: 2 / 13  (15.4%)


/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


15.38

In [19]:
optimizer2 = BootstrapFewShotWithRandomSearch(metric=metric_EM, max_bootstrapped_demos=2, num_candidate_programs=8, num_threads=NUM_THREADS)
rag_compiled = optimizer2.compile(RAG(), trainset=train, valset=dev)

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 8 candidate sets.


Average Metric: 2 / 13  (15.4): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 2270.02it/s]


Average Metric: 2 / 13  (15.4%)
Score: 15.38 for set: [0, 0]
New best score: 15.38 for seed -3
Scores so far: [15.38]
Best score: 15.38


Average Metric: 5 / 13  (38.5): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.74it/s]


Average Metric: 5 / 13  (38.5%)
Score: 38.46 for set: [7, 7]
New best score: 38.46 for seed -2
Scores so far: [15.38, 38.46]
Best score: 38.46


...

Backing off 0.2 seconds after 2 tries calling function <function GPT3.request at 0x1129777e0> with kwargs {}
Backing off 3.3 seconds after 3 tries calling function <function GPT3.request at 0x1129777e0> with kwargs {}


Average Metric: 3 / 10  (30.0):  77%|████████████████████████████████████████████████████████████████████████████████████▌                         | 10/13 [00:07<00:02,  1.14it/s]

Backing off 2.8 seconds after 3 tries calling function <function GPT3.request at 0x1129777e0> with kwargs {}


Average Metric: 4 / 13  (30.8): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:12<00:00,  1.08it/s]

Average Metric: 4 / 13  (30.8%)
Score: 30.77 for set: [7, 7]
Scores so far: [15.38, 38.46, 30.77, 30.77, 30.77, 38.46, 23.08, 30.77, 30.77, 38.46, 30.77]
Best score: 38.46
Average of max per entry across top 1 scores: 0.38461538461538464
Average of max per entry across top 2 scores: 0.6153846153846154
Average of max per entry across top 3 scores: 0.7692307692307693
Average of max per entry across top 5 scores: 0.8461538461538461
Average of max per entry across top 8 scores: 0.9230769230769231
Average of max per entry across top 9999 scores: 1.0
11 candidate programs found.


In [22]:
evaluate_hotpot(rag_compiled)

Average Metric: 5 / 13  (38.5): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 1486.04it/s]

Average Metric: 5 / 13  (38.5%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' 'False' 'False' '✔️ [True]' '✔️ [True]' 'False' 'False'
 'False' 'False' 'False' '✔️ [True]' '✔️ [True]' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,"determine who has a broader scope of profession. We know that E. L. Doctorow was an American novelist, editor, and professor, while Julia Peterkin was...",E. L. Doctorow,✔️ [True]
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida","produce the answer. We know that the song ""Right Back At It Again"" is from the album ""Common Courtesy"" by A Day to Remember. We...","The singer was born in Ocala, Florida.",False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,find the year the party of the winner of the 1971 San Francisco mayoral election was founded. We know that the winner of the election...,1849,False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,"produce the answer. We know that Anthony Dirrell is a professional boxer and the younger brother of Andre Dirrell, who is also a professional boxer....",Andre Dirrell,✔️ [True]
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,"produce the answer. We know that Oliver Cookson is a UK entrepreneur who established the sports nutrition business Myprotein. We also know that in 2011,...",Cheshire,✔️ [True]
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980","find the birth date of the actor. We will first need to identify the actor's name, which is Jo Dong-hyuk. Then, we will need to...","December 11, 1977",False
6,Kyle Moran was born in the town on what river?,Castletown River,"produce the answer. We know that Kyle Moran was born in Dundalk, Ireland and that there are three different pieces of information about people named...",The town of Dundalk is located near the Moran River.,False
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England",produce the answer. We know that the actress who played the niece in the Priest film was Jackie Joseph. We also know that Jackie Joseph...,"Sammie Jacqueline Joseph was born in Los Angeles, California, United States.",False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,"produce the answer. We know that Cathryn Harrison is the daughter of Noel Harrison, who was an actor. We also know that Violet Trefusis was...","The movie is ""The Soul's Gymnasium"".",False
9,What year was the father of the Princes in the Tower born?,1442,determine the year of birth of the father of the Princes in the Tower. We know that the Princes in the Tower were born during...,1441,False


38.46

In [24]:
rag_compiled("What year was the party of the winner of the 1971 San Francisco mayoral election founded?")
turbo.inspect_history(n=1)





Given the fields `context`, `question`, produce the fields `answer`.

---

Question: Who produced the album that included a re-recording of "Lithium"?
Answer: Butch Vig

Question: In what year was the star of To Hell and Back born?
Answer: 1925

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

Question: Which author is English: John Braine or Studs Terkel?
Answer: John Braine

Question: What documentary about the Gilgo Beach Killer debuted on A&E?
Answer: The Killing Season

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Answer: Kevin Greutert

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Answer: Foxcatcher

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «San Diego mayoral election, 1971 | The 1971 San Die

# Multi-Hop Retrieval and Reasoning

In [33]:
from dsp.utils.utils import deduplicate

class MultiHop(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought("question -> search_query")

        self.generate_query_from_context = dspy.ChainOfThought("context, question -> search_query")

        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        passages = []
        
        search_query = self.generate_query(question=question).search_query
        passages += self.retrieve(search_query).passages

        search_query2 = self.generate_query_from_context(context=deduplicate(passages), question=question).search_query

        # TODO: Replace `None` with a call to self.retrieve to retrieve passages. Append them to the list `passages`.
        passages += self.retrieve(search_query2).passages

        return self.generate_answer(context=deduplicate(passages), question=question)

In [29]:
multihop_compiled = optimizer2.compile(MultiHop(), trainset=train, valset=dev)

Average Metric: 4 / 13  (30.8): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  3.25it/s]
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 4 / 13  (30.8%)
Score: 30.77 for set: [0, 0, 0]
New best score: 30.77 for seed -3
Scores so far: [30.77]
Best score: 30.77


Average Metric: 7 / 13  (53.8): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  7.29it/s]


Average Metric: 7 / 13  (53.8%)
Score: 53.85 for set: [7, 7, 7]
New best score: 53.85 for seed -2
Scores so far: [30.77, 53.85]
Best score: 53.85


 43%|█████████████████████████████████████████████████████████████▋                                                                                  | 3/7 [00:08<00:11,  2.79s/it]


...

Backing off 0.4 seconds after 5 tries calling function <function GPT3.request at 0x1129777e0> with kwargs {}


Average Metric: 4 / 10  (40.0):  77%|████████████████████████████████████████████████████████████████████████████████████▌                         | 10/13 [00:25<00:08,  2.92s/it]

Backing off 0.6 seconds after 1 tries calling function <function GPT3.request at 0x1129777e0> with kwargs {}
Backing off 19.3 seconds after 6 tries calling function <function GPT3.request at 0x1129777e0> with kwargs {}


Average Metric: 6 / 13  (46.2): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:48<00:00,  3.75s/it]

Average Metric: 6 / 13  (46.2%)
Score: 46.15 for set: [7, 7, 7]
Scores so far: [30.77, 53.85, 53.85, 53.85, 46.15, 61.54, 46.15, 46.15, 61.54, 69.23, 46.15]
Best score: 69.23
Average of max per entry across top 1 scores: 0.6923076923076923
Average of max per entry across top 2 scores: 0.9230769230769231
Average of max per entry across top 3 scores: 1.0
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
11 candidate programs found.


In [30]:
evaluate_hotpot(multihop_compiled, devset=dev)

Average Metric: 9 / 13  (69.2): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 2084.41it/s]

Average Metric: 9 / 13  (69.2%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' 'False' 'False' '✔️ [True]' '✔️ [True]' 'False' '✔️ [True]'
 'False' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,"determine who has a broader scope of profession. We know that E. L. Doctorow is an American novelist, editor, and professor, while Julia Peterkin is...",E. L. Doctorow,✔️ [True]
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida","find the city where the singer was born. We know that the song ""Right Back At It Again"" is by A Day to Remember, and...",Belleville,False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,"find the year the party was founded. We know that the winner of the 1971 San Francisco mayoral election was Pete Wilson, and we know...",1998,False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,"find the name of the super middleweight title holder. We know that Anthony Dirrell is the younger brother of Andre Dirrell, who is also a...",Andre Dirrell,✔️ [True]
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,find the county where the sports nutrition business is based. We know that the business is called Myprotein and that it was established by Oliver...,Cheshire,✔️ [True]
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980","find the birth date. We know that the actor played roles in First Wives Club and Searching for the Elephant, and we know that her...","January 14, 1971",False
6,Kyle Moran was born in the town on what river?,Castletown River,"find the town where Kyle Moran was born. We know that Kyle Moran is an Irish footballer, and we know that he was born on...",Castletown River,✔️ [True]
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England","find the city and country of birth. We know that the actress is Francesca Cardinale, and we know that she is the niece of Claudia...","Buenos Aires, Argentina",False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,"find the movie. We know that the daughter of Noel Harrison is Cathryn Harrison, and we know that she played Violet Trefusis. We also know...",Portrait of a Marriage,✔️ [True]
9,What year was the father of the Princes in the Tower born?,1442,"find the year of birth. We know that the father of the Princes in the Tower is Edward IV, and we know that he died...",1442,✔️ [True]


69.23

In [32]:
multihop_compiled(question="Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?")
turbo.inspect_history(n=3)





Given the fields `question`, produce the fields `search_query`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the search_query}. We ...
Search Query: ${search_query}

---

Question: In what year was the star of To Hell and Back born?
Reasoning: Let's think step by step in order to find the year the star of To Hell and Back was born. We need to know the name of the star, then we can search for their birth year.
Search Query: To Hell and Back star birth year

---

Question: Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?
Reasoning: Let's think step by step in order to find out who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000. We need to know the name of the team, then we can search for who purchased it in 2000.
Search Query: 1995 Monaco Grand Prix team purchase 2000







Given the fields `context`, `question`, produce the fields `